In [23]:
import json
from mitiq.benchmarks import generate_rb_circuits
from mitiq import zne, ddd, pec, rem

from qiskit_ibm_runtime.fake_provider import FakeJakartaV2 
from functions_to_use import load, validate_experiment, validate_composed_experiment, execute_0s_ideal
from arg_function_maps import schema_to_params, run_composed_experiment

### So let's imagine someone wants to use both DDD and ZNE on their cirucit

In [24]:
zne_experiment = load('./experiments/zne_experiments/zne_experiment_with_technique.json')
ddd_experiment = load('./experiments/ddd_experiments/ddd_experiment_with_technique.json')
print("zne_experiment: \n", json.dumps(zne_experiment, indent=4), "\n")
print("ddd_experiment: \n", json.dumps(ddd_experiment, indent=4))

zne_experiment: 
 {
    "technique": "zne",
    "noise_scaling_factors": [
        2,
        4,
        6,
        8
    ],
    "noise_scaling_method": "global",
    "extrapolation": "richardson",
    "scale_factor": 2
} 

ddd_experiment: 
 {
    "technique": "ddd",
    "rule": "xx"
}


In [42]:
zne_params = schema_to_params(zne_experiment)
ddd_params = schema_to_params(ddd_experiment)

In [26]:
composed_experiment = {"experiments": [ddd_experiment, zne_experiment]}
composition_schema = load('./schema/composition_schema.json')

*** something to think about: right now the $id for the composition_schema is the complete filepath. This is necessary because we are referncing other schema in it (the technique-specific schema)

In [27]:
validate_composed_experiment(composed_experiment, composition_schema)

individual experiment validation passed


In [37]:
n_qubits = 2
depth_circuit = 2
shots = 10 ** 4

circuit = generate_rb_circuits(n_qubits, depth_circuit, trials=1, return_type="qiskit")[0]
execute = execute_0s_ideal

In [38]:
composed_result = run_composed_experiment(composed_experiment, circuit, execute)
print(composed_result)

0.9169999999999968


In [47]:
zne_mitigated = zne.execute_with_zne(circuit, execute, **zne_params)
ddd_mitigated = ddd.execute_with_ddd(circuit, execute, **ddd_params)
unmitigated = execute_0s_ideal(circuit)
ideal = 1 #property of RB circuits

print("ideal = \t \t",ideal)
print("unmitigated = \t \t", "{:.5f}".format(unmitigated))
print("zne mitigated = \t", "{:.5f}".format(zne_mitigated))
print("ddd mitigated = \t", "{:.5f}".format(ddd_mitigated))
print("composed result = \t", "{:.5f}".format(composed_result))

ideal = 	 	 1
unmitigated = 	 	 0.90900
zne mitigated = 	 1.21000
ddd mitigated = 	 0.92500
composed result = 	 0.91700


<br>
<br>
<br>
<br>
<br>
<br>

In [22]:
# ddd_executor = ddd.mitigate_executor(executor=execute, rule=ddd_params['rule'])
# combined_executor_ddd1st = zne.mitigate_executor(executor=ddd_executor, factory=zne_params['factory'], scale_noise=zne_params['scale_noise'])
# combined_result_ddd1st = combined_executor_ddd1st(circuit)

# zne_executor = zne.mitigate_executor(executor=execute, factory=zne_params['factory'], scale_noise=zne_params['scale_noise'])
# combined_executor_zne1st = ddd.mitigate_executor(executor=zne_executor, rule=ddd_params['rule'])
# combined_result_zne1st = combined_executor_zne1st(circuit)

# print("Mitigated value obtained with DDD then ZNE:", "{:.5f}".format(combined_result_ddd1st.real))
# print("Mitigated value obtained with ZNE then DDD:", "{:.5f}".format(combined_result_zne1st.real))